In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [12]:
df_red = pd.read_csv("winequality-red.csv", sep=';',index_col=False)
df_red["type"] = 0
df_red

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,0
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,0
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,0
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,0
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,0
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,0


In [44]:
df_red_1 = df_red.drop(['quality', 'type'], axis=1)
df_red_normalized = pd.DataFrame()
for column in df_red_1.columns:
    media = df_red_1[column].mean()
    std = df_red_1[column].std()

    df_red_normalized[column] = ((df_red_1[column]-media)/std)
matrix_red = df_red_normalized.to_numpy()
cov_red = np.cov(matrix_red,rowvar=False)
eigenvalues, eigenvectors = np.linalg.eig(cov_red)
df_eigenvalues = pd.DataFrame(eigenvalues)
df_eigenvalues = df_eigenvalues.sort_values(by=0, ascending=False)
df_eigenvectors = pd.DataFrame(eigenvectors)
#Pegando apenas o PC1 E PC2
nomes_colunas = df_red_1.columns.tolist()
df_eigenvectors.columns = nomes_colunas
df_eigenvectors = df_eigenvectors[0:2]
df_eigenvectors = df_eigenvectors.rename(index={0:'PC1', 1:'PC2'})
# Print the results
print("Eigenvalues:")
print(df_eigenvalues)
print("\nEigenvectors (columns of the array):")
df_eigenvectors

Eigenvalues:
           0
0   3.099132
1   1.925910
2   1.550543
3   1.213233
4   0.959292
10  0.659608
9   0.583791
8   0.422957
7   0.344642
6   0.181333
5   0.059558

Eigenvectors (columns of the array):


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
PC1,0.489314,-0.110503,-0.123302,-0.229617,-0.082614,-0.639691,-0.249523,0.194021,-0.177595,-0.350227,0.101479
PC2,-0.238584,0.274930,-0.449963,0.078960,0.218735,-0.002389,0.365925,-0.129110,-0.078775,-0.533735,0.411449


In [46]:
#Plota os coeficientes (loadings) para cada componente principal
for i in range(1):
    componente = f"PC{i+1}"
    plt.figure(figsize=(8, 5))
    df_eigenvalues[componente].plot(kind="bar")
    plt.title(f"Loadings das Features na Componente Principal {componente}")
    plt.xlabel("Features originais")
    plt.ylabel(f"Coeficiente em {componente}")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

KeyError: 'PC1'

<Figure size 800x500 with 0 Axes>

In [31]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_red_1)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
loadings = pd.DataFrame(
    pca.components_,
    columns=df_red_1.columns,
    index=['PC1', 'PC2']
)
loadings

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
PC1,0.489314,-0.238584,0.463632,0.146107,0.212247,-0.036158,0.023575,0.395353,-0.438520,0.242921,-0.113232
PC2,-0.110503,0.274930,-0.151791,0.272080,0.148052,0.513567,0.569487,0.233575,0.006711,-0.037554,-0.386181


In [32]:
vec_manual = eigenvectors[:, :2]

# Autovetores do sklearn
vec_sklearn = pca.components_.T  # sklearn coloca PCs nas linhas

# Correlação entre as direções
corr = np.abs(np.round(np.corrcoef(vec_manual.T.flatten(), vec_sklearn.T.flatten())[0,1],3))
print("Correlação entre direções PCA manual vs sklearn:", corr)

Correlação entre direções PCA manual vs sklearn: 1.0


In [13]:
df1 = pd.read_csv("winequality-white.csv", sep=';',index_col=False)
df1['type'] = 1
df1

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6,1
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6,1
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6,1
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,1
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,1
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,1
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,1
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,1


In [15]:
df_tudo = pd.concat([df, df1], ignore_index=True)
df_tudo

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,0
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6492,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6,1
6493,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5,1
6494,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6,1
6495,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7,1
